# RMF Lab 4 week 08 Solutions

This content is authored by Maria Boutchkova for use in the University of Edinbugh Business School research Methods in Finance course in Spring 2023.

Make sure to have covered the material preceeding this Notebook. This notebook goes with the pdf document lab3_instr.pdf

This notebook covers more advanced commands and functions around:

* Dealing with outliers
* Scatterplots
* Running regressions robust to outliers

The first computational cell below (with In [ ] in front) contains the solution. Go over the command lines, make sure they make sense to you, click inside the cell, it should become surrounded by a green rectangle, press Esc - the rectangle will become blue, now press Shift+Enter - this will execute the cell and produce the results beneath it.
While a cell is being executed it shows an asterisk.

To remove all output in the notebook and start again, go to the Kernel tab above, select Restart and Clear Output.

In this notebook we use the functionality of Stata. If you want to explore its full documetation, see here: https://www.stata.com/bookstore/getting-started-windows/

The comment sign in Stata is *

Magics are programs provided by stata_kernel that enhance the experience of working with Stata in Jupyter. Magics start with %. 
The browse magic simply shows us the data in memory (%browse).

The help command allows you to pull stata documentation inside the current notebook, while in regular desktop Stata it opens the Viewer window. Feel free to do this for every command you see used below. 

## Task 1: RESET test

In lab4 we are using the exact same data as in lab3. Import it using the same command as before (import delimited). Examine the variables see the pdf instructions for lab3 for their definitions.

In [ ]:
import delimited BoardGenderPanel.csv, varn(1) clear

In [ ]:
%browse

Task 1.1 We are going to take all the necessary commands from lab3 to quickly run the heteroskedasticity-robust regression. Start with dropping the already existing ln* vars and generate the ones we need once more.

In [ ]:
drop ln*
gen lnTA = ln(ta)
gen lnCEOPay = ln(ceopay)
unab fin_vars: roa - rd_a
unab bd_vars: bgen - ceopay
local drop ta tobinq
local fin_no_ta_tq: list fin_vars - drop
local drop ceopay
local bd_no_pay: list bd_vars - drop
eststo clear
qui eststo: reg tobinq lnTA lnCEOPay `fin_no_ta_tq' `bd_no_pay', vce(cluster gvkey)

Task 1.2 The stata command that computes the RESET test is called estat ovtest. First run it – it has to be done immediately after the regression to which we want it to refer. Then pull the documentation of the estat ovtest command. Note how the test output and the documentation talks about omitted variables – which is imprecise. Remember the RESET test cannot test for omitted variables in general, these is no such statistical test. The RESET test only suggests a different functional form of some x-s might be better.

In [ ]:
estat ovtest

In [ ]:
help estat ovtest

## Task 2: Summary stats

Task 2.1 Save the raw variables involved in the regression in locals and produce summary stats for them using the command tabstat with the option stat(mean sd min median max n). Remember the trick from lab3 on how to keep only the observations that were involved in the regression that was just run. Use the code cell above to check the documentation for tabstat and see what the option means.

In [ ]:
keep if e(sample)
unab fin_vars: roa - rd_a
unab bd_vars: bgen - ceopay
tabstat `fin_vars' `bd_vars', stat(mean sd min median max n) 

Task 2.2 Now produce the same summary stats for the two log-transformed variables involved in the regression.

In [ ]:
tabstat lnTA lnCEOPay, stat(mean sd min median max n) 

Task 2.3 Let us examine some of the outliers using the command list lpermno gvkey datadate ceopay if ceopay < .001. We will then lookup these cases in CapitalIQ.

In [ ]:
list lpermno gvkey datadate ceopay if ceopay < .001